<a href="https://colab.research.google.com/github/fatima448/brain_tumor_detection/blob/main/brain_tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import zipfile
zip_path = "/content/drive/MyDrive/Brain Tumour Detection.v1i.folder.zip"   # CHANGE THIS
extract_dir = "/content/brain_tumor"

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_dir)

print("Extracted to:", extract_dir)


Extracted to: /content/brain_tumor


In [3]:
import os
print(os.listdir("/content/brain_tumor"))


['train', 'valid', 'test', 'README.dataset.txt', 'README.roboflow.txt']


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = 224
BATCH = 32

train_gen = ImageDataGenerator(rescale=1./255)
valid_gen = ImageDataGenerator(rescale=1./255)
test_gen  = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory(
    "/content/brain_tumor/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode="categorical"
)

valid_data = valid_gen.flow_from_directory(
    "/content/brain_tumor/valid",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode="categorical"
)

test_data = test_gen.flow_from_directory(
    "/content/brain_tumor/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode="categorical",
    shuffle=False
)


Found 3858 images belonging to 4 classes.
Found 1099 images belonging to 4 classes.
Found 548 images belonging to 4 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Sequential

base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base.trainable = False

model = Sequential([
    base,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  # 4 classes
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,468 (9.24 MB)

 Trainable params: 164,484 (642.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.6940 - loss: 0.7811 - val_accuracy: 0.8581 - val_loss: 0.3600
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 211s 2s/step - accuracy: 0.8887 - loss: 0.3195 - val_accuracy: 0.8790 - val_loss: 0.3428
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.9106 - loss: 0.2542 - val_accuracy: 0.8926 - val_loss: 0.2556
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.9190 - loss: 0.2119 - val_accuracy: 0.9045 - val_loss: 0.2192
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.9446 - loss: 0.1577 - val_accuracy: 0.8990 - val_loss: 0.2537
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.9331 - loss: 0.1687 - val_accuracy: 0.9181 - val_loss: 0.2024
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.9569 - loss: 0.1270 - val_accuracy: 0.9299 - val_loss: 0.1919
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 207s 2s/step - accuracy: 0.9606 - loss: 0.1038 - val_accu

In [7]:
loss, test_acc = model.evaluate(test_data)
print("Test Accuracy:", test_acc * 100)


18/18 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.8763 - loss: 0.3437
Test Accuracy: 91.24087691307068


In [9]:
model.save("brain_tumor_model.keras")


In [11]:
import gradio as gr
import cv2
import numpy as np
import tensorflow as tf

# Load your model
model = tf.keras.models.load_model("/content/brain_tumor_model.keras")

IMG_SIZE = 224  # same size used during training
CLASS_NAMES = ["glioma", "meningioma", "notumor", "pituitary"]

def predict_tumor(img):
    # Convert to RGB (gradio gives image in RGB)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)[0]
    class_id = np.argmax(pred)
    confidence = float(np.max(pred))

    return {
        "Prediction": CLASS_NAMES[class_id],
        "Confidence": round(confidence * 100, 2)
    }

# Gradio App
interface = gr.Interface(
    fn=predict_tumor,
    inputs=gr.Image(type="numpy"),
    outputs=gr.JSON(),
    title="Brain Tumor Detection",
    description="Upload an MRI scan to detect tumor & identify its type",
)

interface.launch()


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://226e4a3c2f13e76654.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
